# "Building a self driving car - Software Configuration"
> "Setting up software while waiting for the parts to arrive. Lets begin..."
- toc: true
- branch: master
- badges: false
- comments: true
- categories: [self driving, software]
- hide: false
- search_exclude: false
- metadata_key1: MUSHR
- metadata_key2: Software

# Purpose

The intention of this page is NOT to replace the existing documentation by MUSHR. Please follow them [here](https://mushr.io), buit visit this page if things dont go as planned. This is **NOT** a TUTORIAL rather a buffet of practical problems faced and solutions provided to overcome them easily.

![](https://abhisheksreesaila.github.io/blog/images/workflow/sw_process_flowchart.png "Process Flow")


# Scenarios

## Monitor not booting up, after MUSHR image is installed.

**Issue**

When you install the stock image of JETSON NANO using [BelenaEtcher](https://www.balena.io/etcher), connect JETSON NANO using HDMI, NAno boots and you can see the Linux GUI. But, if you do the same exact same thing using [MUSHR IMAGE](https://drive.google.com/file/d/17Xn63ZWA_e3DVxt_ajhzu-uKW0hUG4iX/view?usp=sharing), it does not show the GUI. without the GUI, software config is almost impossible (*well, I am not referring to LINUX gurus here, everything possible from a GUI is also possible with command line in LINUX*). In other words, we need the GUI working.


### Solution 1
**USE DISPLAY PORT**

> Important: If you have a monitor that supports DISPLAY PORT, use it!! Instant solution. (Time saving : 3-4 days)

**If you only have HDMI port, read on**

### Solution 2
**USE COMMAND LINE**


#### STEP 1: SSH

You need to LOGIN (using SSH). For SSH, you need WIFI, For WIFI you need to LOGIN. Get it! chicken-egg problem? Almost. There is one possible solution, **Hard-wired USB connection**

- Connect the **MICRO USB** port of NANO to the **USB PORT** of your laptop (also known as HEADLESS LOGIN)

**HEADLESS LOGIN (on a Mac)**

```bash

 #Go to dev directory
cd /dev  

 #check for all things like tty and usb
ls *tty*usb*       
tty.usbmodem146123  # result (yours might be slightly different)

 #use the screen command with the result of previous step + serial interface number
screen tty.usbmodem146123 115200  #  115200 is the Jetson serial interface. "screen" is installed on a mac by default

# (It will ask for SUDO login and password)

```

![](https://abhisheksreesaila.github.io/blog/images/software-config/screen-login-password.png "login to nano via SCREEN command via Micro-USB")



**ACTIVATE THE ROOT in JETSON NANO**

Be aware that the root account is not active by default. You will need to do:

```bash
sudo passwd root
```

Then go ahead to set password. Use the root user using

```bash
su root
```

**USE WPA_SUPPLICANT TO CONNECT** 

```bash

# CREATE a file at */etc/wpa_supplicant/wpa_supplicant.conf*

vi /etc/wpa_supplicant/wpa_supplicant.conf

# PRESS "I" to start insert mode  

# Add this content to the file

ctrl_interface=/run/wpa_supplicant
update_config=1

# PRESS ":wq" to save and quit the editor

# Then, run this command

wpa_supplicant -B -i wlan0 -c /etc/wpa_supplicant/wpa_supplicant.conf

wpa_cli -i wlan0

# This will present an interactive prompt (>), which has tab completion and descriptions of completed commands.

scan

# Ok
# <3>CTRL-EVENT-SCAN-RESULTS

scan_results

# bssid / frequency / signal level / flags / ssid
# 00:00:00:00:00:00 2462 -49 [WPA2-PSK-CCMP][ESS] MYSSID
# 11:11:11:11:11:11 2437 -64 [WPA2-PSK-CCMP][ESS] ANOTHERSSID

add_network
# 0

set_network 0 ssid "MYSSID"

set_network 0 psk "passphrase"

enable_network 0

#<2>CTRL-EVENT-CONNECTED - Connection to 00:00:00:00:00:00 completed (reauth) [id=0 id_str=]

save_config
# OK

quit
 
```


**Check if the wireless device is up**

```bash
ip link show wlan0    #Look for the word "UP" inside the brackets in the first line of the output.

/sbin/iw wlan0 link  # Connected to <your network>>

ping google.com    # testing if it connects

ifconfig  #GET the IP. Needed to connect to SSH. Uff, finally!

```

**Connect to the JETSON NANO

Open a terminal window

```bash
ssh -X robot@IP_ADDRESS  #enter your IP address (Notice -X)

```



#### STEP 2: GPARTED, BLDC, BLUETOOTH, IP ADDRESS FIX

Before installing anything, there seems to be some sort of lock on the "linux" installer to donwload. For eg.

*E: Could not get lock /var/lib/dpkg/lock – open (11: Resource temporarily unavailable) E: Unable to lock the administration directory (/var/lib/dpkg/), is another process using it?*

Here is the fix : 

```bash

sudo rm /var/lib/apt/lists/lock
sudo rm /var/cache/apt/archives/lock
sudo rm /var/lib/dpkg/lock

```


**GPARTED**

```bash

sudo apt-get install gparted

gparted # opens up the GUI as shown below. Now, follow instructions from the MUSHR site. 

```

![](https://abhisheksreesaila.github.io/blog/images/software-config/gparted.png "gparted GUI opened via SSH")

![](https://abhisheksreesaila.github.io/blog/images/software-config/gparted-2.png "gparted GUI opened via SSH")


**BLUETOOTH**

```bash

hcitool dev  

```

If the Bluetooth adapter is working and available, you should see its address listed (each address is unique):

*Devices:
	hci0	38:##:##:##:##:##*

```bash 

bluetoothctl    #opens up a bluetooth prompt

agent on
scan on  # scan near by bluetooth devices. Start your PS4 controller
scan off # Switch it off after getting the MAC ADDRESS of the wireless controller
trust MAC_ADDRESS   # trust, pair and connect 
pair MAC_ADDRRESS
connect MAC_ADDRESS

```

![](https://abhisheksreesaila.github.io/blog/images/software-config/bluetooth-pair.png "command line bluetooth pairing")

After connecting to the bluetooth, we should test it. 

```bash

sudo apt-get install -y jstest-gtk   # install the jstest package

jstest-gtk    #run this. It will open up GUI as follows

```

![](https://abhisheksreesaila.github.io/blog/images/software-config/bluetooth-controller.png "bluetooth-controller")

**IP ADDRESS FIX**

ROS IP and MASTER URI will *not* be set to your IP automatically. USe this commands below in the local bashrc file so that it is set everytime you login (vs you typing these commands everytime)

```bash
export ROS_IP=<<YOUR_IP_ADDRESS>>
export ROS_MASTER_URI=http://<<YOUR_IP_ADDRESS>>:11311

```

![](https://abhisheksreesaila.github.io/blog/images/software-config/ROS-IP-SET.png "Set the ROS_IP and ROS_MASTER_URI")



**BLDC Tool**

> Important: Now, this step can only be done when you finish the Hardware configuration since you need the VESC connected to JETSON NANO + MOTOR to test it properly. 

```bash

 cd && ./bldc-tool/BLDC_Tool
 
```
 

If you have ordered **Flipsky VESC**, chances are that you will have 3.X firmware installed (*The hardware version is 410/411/412, but the firmware version is different*). The problem is that BLDC tool can only support 2.18 (and hence MUSHR car too!). This wil throw an SERIAL PORT ERROR and will not connect to BLDC TOOL GUI. BLDC is old tool and is replaced by [**VESC TOOL**](https://vesc-project.com)

- Download the VESC TOOL from the website and it will let you connect easily.
- It will ask you to install the latest software.**DO NOT** install latest.
- Use "custom" firmware (follow this [link](https://vesc-project.com/node/269)). The custom firmware is nothing but the BLDC servoout.bin file that is installed via BLDC tool. (also available [here](https://github.com/vedderb/bldc-tool/tree/master/firmwares/hw_410_411_412)) 



![](https://abhisheksreesaila.github.io/blog/images/software-config/bldc-firmware-to-install.png "VESC TOOL CUSTOM FIRMWARE TO BE USED")

![](https://abhisheksreesaila.github.io/blog/images/software-config/vesc-tool.png "VESC TOOL")


- Now, open the BLDC tool and hit "connect" and it will be successful. Now IF VESC tool is a newer version of the BLDC tool, why is heck do we need BLDC tool? MUSHR parameters has been set to BLDC tool  and works only when you use it. After your gain mastery over the MUSHR software you can try VESC tool.

- Follow the instructions from the MUSHR.IO website i.e load the sensorless xml config file and "write" it back to VESC

- Push the "top" arrow key and the wheels should move forward, and vice versa, if its moves in the reverse direction, it means the "wires" from the VESC to the motor has been criss crossed. reverse the wire connection and you should be good. See below for visual. 

![](https://abhisheksreesaila.github.io/blog/images/software-config/vesctomotor_connection.png "VESC TO MOTOR CONNECTION")

# References
 - [wiki.archlinux](https://wiki.archlinux.org/index.php/Wpa_supplicant)
 - [bbs.archlinux.org](https://bbs.archlinux.org/viewtopic.php?id=234877)
 - [linuxcommando](https://linuxcommando.blogspot.com/2013/10/how-to-connect-to-wpawpa2-wifi-network.html)
 - [desertbot](https://desertbot.io/blog/how-to-add-a-dual-wifi-bluetooth-card-to-a-jetson-nano-intel-8265)
    